In [ ]:
!pip install playwright
!python3 -m playwright install
from playwright.async_api import async_playwright

!pip install musicbrainzngs
import musicbrainzngs as mbz
from musicbrainzngs.musicbrainz import WebServiceError
mbz.set_useragent("My App", "1.0", "my.email@example.com")  # random placeholder user agent

!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up your credentials
client_id = '6675a87b504d4558ad3cf69845f31ec5'
client_secret = 'a075360fc30b4c3ea8cdb70e90af48d4'

# Authenticate using Client Credentials Flow
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libavif.so.13                                    ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.11/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:269:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:105

In [ ]:
# Get the top listeners from a given album

# Async function to extract Top Listeners' data-text from the <a> tag inside the .listener div
async def get_top_listeners_data_text(url):
    async with async_playwright() as p:
        # Launch the browser in headless mode
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Navigate to the album page
        await page.goto(url)

        # Wait for the page to load completely (wait for the 'top-listeners' section)
        await page.wait_for_selector('.top-listeners')

        # Extract all 'listener' divs within the 'top-listeners' section
        listeners = await page.query_selector_all(".top-listeners .listener")

        listener_names = set()

        if listeners:
            #print("Top Listeners' data-text:")
            for listener in listeners:
                # Find the <a> tag inside the listener div
                a_tag = await listener.query_selector('a')

                # Get the 'data-text' attribute from the <a> tag
                if a_tag:
                    data_text = await a_tag.get_attribute('data-text')
                    if data_text:
                        #print(f"- {data_text}")
                        listener_names.add(data_text)
                    else:
                        print("- No data-text found in <a> tag.")
                else:
                    print("- No <a> tag found inside listener.")
        else:
            print("No listeners found.")

        # Close the browser
        await browser.close()

        return listener_names

In [ ]:
from playwright.async_api import async_playwright

# Async function to extract all MusicBrainz recording links from a user's recommendations page
async def get_listenbrainz_recording_links(user):
    async with async_playwright() as p:
        print(user)
        # Launch the browser in headless mode
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Go to the user's recommendations page
        url = f"https://listenbrainz.org/user/{user}/recommendations/"
        await page.goto(url)

        try:
            await page.wait_for_selector('.playlist-items')
            playlist_items = await page.query_selector_all('.playlist-items a[href^="https://musicbrainz.org/recording/"]')

            if playlist_items:
                songs = {await item.get_attribute('href') for item in playlist_items}
            else:
                print(f"No MusicBrainz links found for user {user}.")
                songs = set()

        except Exception as e:
            print(f"Error getting recommendations for user {user}: {e}")
            songs = set()

        # Close the browser
        await browser.close()

        return songs

In [ ]:
def get_id_from_url(url):
  return url.rstrip('/').split('/')[-1]

In [ ]:
def get_song_info(id):
  try:
    song = mbz.get_recording_by_id(id, includes=['artist-credits'])
    #print(song)
    title = song['recording']['title']
    artist = song['recording']['artist-credit'][0]['artist']['name']
  except WebServiceError as exc:
    print("Something went wrong with the request: %s" % exc)

  return title, artist

In [ ]:
def get_album_artist(id):
  try:
    album = mbz.get_release_group_by_id(id, includes=['artist-credits'])
    artist = album['release-group']['artist-credit'][0]['artist']['name']
  except WebServiceError as exc:
    print("Something went wrong with the request: %s" % exc)

  return artist


def get_artists_from_album_urls(urls):
  artists = set()
  for url in urls:
    artists.add(get_album_artist(get_id_from_url(url)))

  return artists

In [ ]:
async def get_top_listeners_of_all_albums(urls):
  async with async_playwright() as p:
    print("Getting Top Listeners of Albums")
    top_listeners = set()
    for url in urls:
      names = await get_top_listeners_data_text(url)
      top_listeners.update(names)

    return top_listeners


async def get_recommended_songs_of_listeners(users):
  async with async_playwright() as p:
    # get recommended songs for top listeners
    recommended_songs = set()
    for user in users:
      playlist = await get_listenbrainz_recording_links(user)
      recommended_songs.update(playlist)

    return recommended_songs

In [ ]:
# Search for track by song name and artist
def get_spotify_url(song_name, artist_name):
    query = f'track:{song_name} artist:{artist_name}'
    result = sp.search(q=query, type='track', limit=1)

    tracks = result.get('tracks', {}).get('items', [])
    if tracks:
        track = tracks[0]
        return track['external_urls']['spotify']
    else:
        print("No track found.")


def get_spotify_popularity(spotify_id):
    return sp.track(spotify_id)['popularity']

In [ ]:
async def add_song_info(song_urls):
  recommendations = list()  # Initialize outside the async with block
  async with async_playwright() as p:
    for song_url in song_urls:
      try:  # Wrap in try-except for individual song errors
        title, artist = get_song_info(get_id_from_url(song_url))
        spotify_url = get_spotify_url(title, artist)
        if spotify_url is None:
          popularity = -1
        else:
          popularity = get_spotify_popularity(get_id_from_url(spotify_url))
        info = {'title': title, 'artist': artist, 'spotify_url': spotify_url, 'popularity': popularity}
        recommendations.append(info)
        print(info)
      except Exception as e:
        print(f"Error processing song {song_url}: {e}")

  return recommendations  # Return outside the async with block

In [ ]:
# Run Me

import asyncio

# Paste in ListenBrainz urls to album here
# example, https://listenbrainz.org/album/9f7a4c28-8fa2-3113-929c-c47a9f7982c3/ is Sgt Pepper's Lonely Hearts Club Band by The Beatles
# While you can input multiple albums, recommended to only input 1, as each album takes about ten minutes each
album_urls = ['https://listenbrainz.org/album/9f7a4c28-8fa2-3113-929c-c47a9f7982c3/']

# get the top listeners for that album from listenbrainz
users = await get_top_listeners_of_all_albums(album_urls)

# compile list of recommended songs for those users
songs = await get_recommended_songs_of_listeners(users)


# Get song title and artist from musicbrainz
# Get spotify url for song from Spotify API
try:
    recommendations = await add_song_info(songs)
except asyncio.CancelledError:
    print("Task was cancelled.")
except Exception as e:
    print(f"An error occurred: {e}")

# gets artists from initial input albums
input_artists = get_artists_from_album_urls(album_urls)

# Remove songs made by the same artists from the input albums
# Remove songs without a spotify url
filtered_recommendations = [song for song in recommendations if song['artist'] not in input_artists and song['spotify_url'] is not None]
# Sort songs by popularity, least popularity at the front
filtered_recommendations = sorted(filtered_recommendations, key=lambda x: x['popularity'])

Getting Top Listeners of Albums
allie-wake-up
ptaff
afestein
duncethethird


ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


ajayijoshua8
diegopostmodern
hugo141592
kozumel
keir2
{'title': 'Lord, Save Me From Myself', 'artist': 'Jon Foreman', 'spotify_url': 'https://open.spotify.com/track/78KPLwnHDO0q4mkgkUVyNX', 'popularity': 26}
{'title': 'Casket Pretty', 'artist': 'Noname', 'spotify_url': 'https://open.spotify.com/track/40myXrYPCBaYOt7AqBdoFO', 'popularity': 38}
{'title': 'Run Through the Jungle', 'artist': 'Creedence Clearwater Revival', 'spotify_url': 'https://open.spotify.com/track/5VfgDw3oOzcNoszDwkLQhH', 'popularity': 72}
{'title': 'St. Chroma', 'artist': 'Tyler, The Creator', 'spotify_url': 'https://open.spotify.com/track/3tFed7YsjGnIfxeLEQwx3R', 'popularity': 88}
{'title': 'Nobody to Love', 'artist': '13th Floor Elevators', 'spotify_url': 'https://open.spotify.com/track/6rykMHc94zgo7HAUqGtvlX', 'popularity': 20}
{'title': 'It’s Too Late', 'artist': 'Carole King', 'spotify_url': 'https://open.spotify.com/track/12q3V8ShACq2PSWINMc2rC', 'popularity': 74}
{'title': 'How Much a Dollar Cost', 'artist': '

In [ ]:
filtered_recommendations

NameError: name 'filtered_recommendations' is not defined